In [1]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import random
import time
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import pandas as pd
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [25]:
def get_pdf_text(pdf_path):
    text = ""
    if pdf_path.endswith('.pdf'):
        # Process PDF file
        pdf_reader = PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

a = get_pdf_text("docs/Panduan Penerimaan Mahasiswa .pdf")
print(a)


Panduan Penerimaan Mahasiswa Baru
Pemasaran dan admisi ITPLNLantai 2 Mezzanine SelatanMenara PLN Jl. Lingkar Luar Barat Duri KosambiCengkareng, Jakarta Barat 11750Telp: 021-5440342 ext 2204
Sambutan
INSTITUT TEKNOLOGI PLNYour Future Starts Today!
INTERNASIONAL, MODEREN, MANDIRI, UNGGULKamiatasnamaInstitutTeknologiPLN(ITPLN)mengucapkanselamatdatangdanterimakasihataskepercayaansaudara/itelahmemilihInstitutTeknologiPLNsebagaisalahsatutujuanuntukmelanjutkanstudi.BukupanduaninimembantusaudaradalammelakukanprosespendaftarandanregistrasiuntukmenjadicalonmahasiswaInstitutTeknologiPLN.Secarasingkat,bukupanduaniniberisitentanginformasiumumdanpedomanyangmenyangkutpendaftaran,jalurpenerimaanmahasiswabaru,ketentuanregistrasi,ketentuanpengundurandiri.InstitutTeknologiPLNakanberkomitmenuntukmengantarkanmahasiswamenjadiinsanakademikyangunggul,moderndanMandiridalambidangTeknolgiyangberwawasanlingkungansesuaidenganVisiITPLNmelaluiprosespendidikanyangdilaksanakan.Sekiandan TerimaKasih.RektorITPLNProf. Dr

In [27]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

chunk = get_text_chunks(a)
print (chunk)

['Panduan Penerimaan Mahasiswa Baru\nPemasaran dan admisi ITPLNLantai 2 Mezzanine SelatanMenara PLN Jl. Lingkar Luar Barat Duri KosambiCengkareng, Jakarta Barat 11750Telp: 021-5440342 ext 2204\nSambutan\nINSTITUT TEKNOLOGI PLNYour Future Starts Today!\nINTERNASIONAL, MODEREN, MANDIRI, UNGGULKamiatasnamaInstitutTeknologiPLN(ITPLN)mengucapkanselamatdatangdanterimakasihataskepercayaansaudara/itelahmemilihInstitutTeknologiPLNsebagaisalahsatutujuanuntukmelanjutkanstudi.BukupanduaninimembantusaudaradalammelakukanprosespendaftarandanregistrasiuntukmenjadicalonmahasiswaInstitutTeknologiPLN.Secarasingkat,bukupanduaniniberisitentanginformasiumumdanpedomanyangmenyangkutpendaftaran,jalurpenerimaanmahasiswabaru,ketentuanregistrasi,ketentuanpengundurandiri.InstitutTeknologiPLNakanberkomitmenuntukmengantarkanmahasiswamenjadiinsanakademikyangunggul,moderndanMandiridalambidangTeknolgiyangberwawasanlingkungansesuaidenganVisiITPLNmelaluiprosespendidikanyangdilaksanakan.Sekiandan TerimaKasih.RektorITPLNPr